In [15]:
import numpy as np
import json
import matplotlib.pyplot as plt
import os

In [20]:
def load_runs(experiment_folder, run_filter, results_path="results_synth_data/"):
    """
    Loads all runs from a specific folder that match a filter.

    Args:
        experiment_folder (str): The name of the folder inside results_path (e.g., 'Forrester_EI_OCBO').
        run_filter (str): A string to filter the run directories inside the experiment_folder (e.g., 'eta09').
        results_path (str, optional): The root path for results. Defaults to "../results_synthetic_data/".
    """
    runs = []
    # Construct the full path to the experiment directory
    experiment_path = os.path.join(results_path, experiment_folder)
    
    if not os.path.isdir(experiment_path):
        print(f"Directory not found: {experiment_path}")
        return runs

    for dir_name in os.listdir(experiment_path):
        if run_filter in dir_name:
            # Load metrics
            metrics_path = os.path.join(experiment_path, dir_name, "metrics.json")
            if not os.path.exists(metrics_path):
                continue
            with open(metrics_path) as f:
                metrics = json.load(f)
            # Load dataset
            dataset_path = os.path.join(experiment_path, dir_name, "dataset.json")
            if not os.path.exists(dataset_path):
                continue
            with open(dataset_path) as f:
                dataset = json.load(f)
            runs.append({"metrics": metrics, "dataset": dataset})
    
    # Rescale regret
    for run in runs:
        if "y_std" in run["dataset"]:
            y_std = run["dataset"]["y_std"]
            run["metrics"]["y_regret_pool_rescaled"] = np.array(run["metrics"]["y_regret_pool"]) * y_std
        
    # Extract calibration
    for run in runs:
        run["metrics"]["y_calibration_mse"] = np.array(run["metrics"]["y_calibration_mse"])
        run["metrics"]["p_array"] = np.array(run["metrics"]["p_array"])
        
    print(f"Loaded {len(runs)} runs from '{experiment_folder}' matching '{run_filter}'.")
    return runs

In [21]:
def process_runs(runs):
    """
    Calculates average and standard deviation for key metrics directly.
    """
    if not runs:
        return (None, None, None, None, None, None, None, None)

    def _get_metric_stats(metric_key):
        """Helper to safely extract and process a single metric."""
        metric_data = [np.array(run["metrics"].get(metric_key, [])) for run in runs]
        metric_data = [m for m in metric_data if m.size > 0]
        if not metric_data:
            return None, None
        
        min_len = min(len(m) for m in metric_data)
        metric_data = [m[:min_len] for m in metric_data]
        
        return np.mean(metric_data, axis=0), np.std(metric_data, axis=0)

    avg_regret, std_regret = _get_metric_stats("y_regret_pool_rescaled")
    avg_cal_mse, std_cal_mse = _get_metric_stats("y_calibration_nmse")
    avg_neg_entropy, std_neg_entropy = _get_metric_stats("mean_negative_entropy")
    avg_sharpness, std_sharpness = _get_metric_stats("uncertainty_sharpness_(toolbox)")

    return (avg_regret, std_regret, avg_cal_mse, std_cal_mse, 
            avg_neg_entropy, std_neg_entropy, avg_sharpness, std_sharpness)

In [26]:
ackley_ei_ocbo = load_runs(
    experiment_folder="Ackley_EI",
    run_filter="ocbo"
)

ackley_ei_bo = load_runs(
    experiment_folder="Ackley_EI",
    run_filter="_bo"
)

Loaded 5 runs from 'Ackley_EI' matching 'ocbo'.
Loaded 5 runs from 'Ackley_EI' matching '_bo'.


In [27]:
(avg_regret_ackley_ocbo, avg_std_regret_ackley_ocbo,
 avg_cal_mse_ackley_ocbo, avg_std_cal_mse_ackley_ocbo,
 avg_neg_entropy_ackley_ocbo, avg_std_neg_entropy_ackley_ocbo,
 avg_sharpness_ackley_ocbo, avg_std_sharpness_ackley_ocbo) = process_runs(ackley_ei_ocbo)

(avg_regret_ackly_bo, avg_std_regret_ackly_bo,
 avg_cal_mse_ackly_bo, avg_std_cal_mse_ackly_bo,
 avg_neg_entropy_ackly_bo, avg_std_neg_entropy_ackly_bo,
 avg_sharpness_ackly_bo, avg_std_sharpness_ackly_bo) = process_runs(ackley_ei_bo)
